In [1]:
df = pd.read_csv('preprocessed_foodrecipes.csv')
df.head(1)

NameError: name 'pd' is not defined

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('preprocessed_foodrecipes.csv')
df.head(1)

,link,title,ratings,num_reviews,ingredients,servings,cook_time,calorie,fat_g,carb_g,protein_g,cholesterol_mg,sodium_mg
0,https://www.allrecipes.com/recipe/100164/chip-...,Chip Truck Fries,4.44,44,"['6 baking potatoes, cut into 1/4 inch slices'...",4.0,60,369.0,14.0,56.3,6.5,0.0,478.0


In [4]:
df.shape

(4776, 13)

In [5]:
df = df[['title', 'ingredients']]
df.ingredients[0]

"['6 baking potatoes, cut into 1/4 inch slices', '1/4 cup vegetable oil', '2 teaspoons seasoned salt']"

In [6]:
import ast

def list_to_str(x):
    x = ast.literal_eval(x)
    x = ','.join(x)
    return x

In [7]:
df.ingredients = df.ingredients.apply(list_to_str)
df.ingredients[0]

'6 baking potatoes, cut into 1/4 inch slices,1/4 cup vegetable oil,2 teaspoons seasoned salt'

In [8]:
data = df.ingredients.values.tolist()
data = [re.sub(r'[^a-zA-Z]', ' ', sent.lower()) for sent in data] 
data[0]

NameError: name 're' is not defined

In [9]:
import re
data = df.ingredients.values.tolist()
data = [re.sub(r'[^a-zA-Z]', ' ', sent.lower()) for sent in data] 
data[0]

'  baking potatoes  cut into     inch slices     cup vegetable oil   teaspoons seasoned salt'

In [10]:
units = ['spoon','pound','cup','quart','ounce','pint','degree','dash','pinch','pack','halve','bunch','inch']
for unit in units:
    expre = re.compile('\w*' + unit + '\w*')
    data = [re.sub(expre, '', sent).strip() for sent in data]

data[0]

'baking potatoes  cut into      slices      vegetable oil    seasoned salt'

In [11]:
data = [' '.join(sent.split()) for sent in data]

data[0]

'baking potatoes cut into slices vegetable oil seasoned salt'

In [12]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel, LdaMulticore, CoherenceModel

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(sentence))

data_words = list(sent_to_words(data))
data_words[0]

ModuleNotFoundError: No module named 'gensim'

In [13]:
pip install -U gensim

  Created wheel for smart-open: filename=smart_open-2.1.0-py3-none-any.whl size=110324 sha256=047501a5e0218f7c8fd58b04e2da20a00e9049f50fdf4d246d97e4e3fa2bee1c
  Stored in directory: c:\users\chamo\appdata\local\pip\cache\wheels\56\b5\6d\86dbe4f29d4688e5163a8b8c6b740494310040286fca4dc648
  Created wheel for boto3: filename=boto3-1.14.39-py2.py3-none-any.whl size=127854 sha256=84fe2eaedc38db4ce5951fc4cb023f5e58b536ee1ae37fae4f0e44c386b2a8c7
  Stored in directory: c:\users\chamo\appdata\local\pip\cache\wheels\94\45\60\7636b61fcfc88fff997a248cd9e1318f3e01533a91aaca552e
Successfully built smart-open boto3
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.15
    Uninstalling Cython-0.29.15:
      Successfully uninstalled Cython-0.29.15
Note: you may need to restart the kernel to use updated packages.


In [14]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel, LdaMulticore, CoherenceModel

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(sentence))

data_words = list(sent_to_words(data))
data_words[0]

['baking',
 'potatoes',
 'cut',
 'into',
 'slices',
 'vegetable',
 'oil',
 'seasoned',
 'salt']

In [15]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 

In [16]:
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [17]:
data_bigram = [bigram_mod[doc] for doc in data_words]
data_bigram[0]

['baking',
 'potatoes',
 'cut',
 'into',
 'slices',
 'vegetable',
 'oil',
 'seasoned',
 'salt']

In [18]:
data_bigram[1]

['irish',
 'bacon',
 'diced',
 'large',
 'potatoes',
 'peeled',
 'and',
 'cubed',
 'can',
 'diced',
 'tomatoes',
 'with',
 'juice',
 'chicken',
 'stock',
 'or',
 'as',
 'needed',
 'salt',
 'and',
 'black',
 'pepper',
 'to',
 'taste',
 'thinly',
 'sliced',
 'dark',
 'green',
 'savoy',
 'cabbage',
 'leaves']

In [19]:
stop_words = stopwords.words('english')

NameError: name 'stopwords' is not defined

In [20]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [21]:
data_nonstop = [[word for word in doc if word not in stop_words] for doc in data_bigram]
data_nonstop[0]

['baking', 'potatoes', 'cut', 'slices', 'vegetable', 'oil', 'seasoned', 'salt']

In [22]:
data_nonstop[1]

['irish',
 'bacon',
 'diced',
 'large',
 'potatoes',
 'peeled',
 'cubed',
 'diced',
 'tomatoes',
 'juice',
 'chicken',
 'stock',
 'needed',
 'salt',
 'black',
 'pepper',
 'taste',
 'thinly',
 'sliced',
 'dark',
 'green',
 'savoy',
 'cabbage',
 'leaves']

In [23]:
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatization(texts, allowed_postages=['NOUN','ADJ']):
    texts_out = []
    for sent in texts:
        doc = nlp(' '.join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postages])
    return texts_out

In [24]:
import spacy

ModuleNotFoundError: No module named 'spacy'

In [25]:
pip install spacy

  Created wheel for wasabi: filename=wasabi-0.7.1-py3-none-any.whl size=20841 sha256=9ad6a3b4bfabde7efe76c1ba00cc6dd0954a5463cc2d36801e3dea26d2c3ad1e
  Stored in directory: c:\users\chamo\appdata\local\pip\cache\wheels\dc\5e\d4\727b6213e9ebec502ff1bf5998f4a83fef87c3aace8a492243
Successfully built wasabi
Note: you may need to restart the kernel to use updated packages.


In [26]:
import spacy

In [30]:
nlp = spacy.load('en', disable=['parser', 'ner'])

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [28]:
python -m spacy download en

SyntaxError: invalid syntax (<ipython-input-28-fc4d5d118d23>, line 1)

In [31]:
nlp = spacy.load('en', disable=['parser', 'ner'])

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [32]:
data_lemmatized = lemmatization(data_nonstop, allowed_postags=['NOUN','ADJ'])
data_lemmatized[0]

TypeError: lemmatization() got an unexpected keyword argument 'allowed_postags'